In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl

from torch import Tensor
from lightly.models import utils
from typing import Optional, List,Tuple, Dict
from sklearn.metrics import roc_auc_score, average_precision_score

In [2]:
class EvaluationModel(pl.LightningModule):
    def __init__(self,
                backbone:nn.Module,
                learning_rate: float =  1e-3,
                weight_decay: float = 0.0,
                output_dim: int = 14,
                freeze: bool = False,
                max_epochs: int = 50,
                mask_ratio: float = 0.15,
                ) -> None:
        super().__init__()

        # self.save_hyperparameters() 
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.max_epochs = max_epochs
        self.output_dim = output_dim
        self.mask_ratio = mask_ratio
        self.backbone = backbone
        
        self.train_step_preds = []
        self.train_step_label = []
        
        self.val_step_preds = []
        self.val_step_label = []
        
        self.test_step_preds = []
        self.test_step_label = []
        
     
        if freeze:
            utils.deactivate_requires_grad(self.backbone)
        else:
            utils.activate_requires_grad(self.backbone)
               
    def forward(self,
                x: Tensor
               ) -> Tensor:
        x = self.backbone(x)
        x = torch.sigmoid(x)
        return x
    
    

    def training_step(self, 
                      batch: List[Tensor], 
                      batch_idx: int
                     ) -> float:
        
        input, label = batch
        prediction = self.forward(input)
        
        self.train_step_label.append(label)
        self.train_step_preds.append(prediction)

        loss = nn.BCELoss()(prediction, label)            
        self.log("train_loss", loss, on_epoch= True,on_step=False , logger=True,)
        
        return {'loss':loss,
                'pred':prediction,
                'label':label}

    def on_train_epoch_end(self) -> None:


        
        y = torch.cat(self.train_step_label).detach().cpu()
        pred = torch.cat(self.train_step_preds).detach().cpu()

        auroc = np.round(roc_auc_score(y, pred), 4)
        auprc = np.round(average_precision_score(y, pred), 4)   
        self.log('train_auroc',auroc, on_epoch=True, on_step=False,logger=True)
        self.log('train_auprc',auprc, on_epoch=True, on_step=False,logger=True)      
        self.train_step_label.clear()
        self.train_step_preds.clear()
        
    def validation_step (self, 
                      batch: List[Tensor], 
                      batch_idx: int
                     ) -> float:
        
        input,label = batch
        prediction = self.forward(input) 
        
        self.val_step_label.append(label)
        self.val_step_preds.append(prediction)

        loss = self._bce_loss(prediction, label,mode='val')       
        self.log("val_loss", loss, on_epoch= True,on_step=False,logger=True)

        return {'loss':loss,
                'pred':prediction,
                'label':label}

    def on_validation_epoch_end(self,*arg, **kwargs) -> None:
        
        y = torch.cat(self.val_step_label).detach().cpu()
        pred = torch.cat(self.val_step_preds).detach().cpu()

#         i=0
#         for output in outputs:
#             if i==0:
#                 y = output['label'].detach().cpu()
#                 pred = output['pred'].detach().cpu()
#             else:
#                 y = torch.cat((y, output['label'].detach().cpu()))
#                 pred = torch.cat((pred, output['pred'].detach().cpu()))
#             i+=1

        auroc = np.round(roc_auc_score(y, pred), 4)
        auprc = np.round(average_precision_score(y, pred), 4)   
        self.log('val_auroc',auroc, on_epoch=True, on_step=False, logger=True)
        self.log('val_auprc',auprc, on_epoch=True, on_step=False, logger=True)    
        self.val_step_label.clear()
        self.val_step_preds.clear()
        
    def test_step(self, 
                  batch: List[Tensor], 
                  batch_idx: int
                 ) -> float:
        input, label = batch
        prediction = self.forward(input)
        
        self.test_step_label.append(label)
        self.test_step_preds.append(prediction)
        
        loss = self._bce_loss(prediction, label,mode='test')
        self.log("test_loss", loss, on_epoch= True,on_step=False , logger=True)

        
        return {'loss':loss,
                'pred':prediction,
                'label':label}

    def on_test_epoch_end(self,*arg, **kwargs) -> None:
        y = torch.cat(self.test_step_label).detach().cpu()
        pred = torch.cat(self.test_step_preds).detach().cpu()

#         i=0
#         for output in outputs:
#             if i==0:
#                 y = output['label'].detach().cpu()
#                 pred = output['pred'].detach().cpu()
#             else:
#                 y = torch.cat((y, output['label'].detach().cpu()))
#                 pred = torch.cat((pred, output['pred'].detach().cpu()))
#             i+=1

        auroc = np.round(roc_auc_score(y, pred), 4)
        auprc = np.round(average_precision_score(y, pred), 4)   
        self.log('test_auroc',auroc, on_epoch=True, on_step=False, logger=True)
        self.log('test_auprc',auprc, on_epoch=True, on_step=False, logger=True)   
        
        self.test_step_label.clear()
        self.test_step_preds.clear()

    def configure_optimizers(self):

        optimizer = optim.Adam(params=self.parameters(), 
                                   lr=self.learning_rate, 
                                   weight_decay=self.weight_decay
                                   )

        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer,
                                                         eta_min=0,
                                                         T_max=self.max_epochs
                                                         )
        
        return {'optimizer': optimizer,
                'lr_scheduler': scheduler
               }
    

    
    def _bce_loss(self, preds, y,mode='train'):
        loss = nn.BCELoss()(preds, y)
        if torch.is_tensor(y):
            y = y.detach().cpu().numpy()
        return loss

In [3]:
import os
import glob
import torch
import numpy as np
import pandas as pd 
import pytorch_lightning as pl
import torchvision.transforms as T

from PIL import Image
from typing import Optional
from torch.utils.data import Dataset, DataLoader

In [4]:
def preprocess(data_dir:str,
               paths: list,
               split:str,
               ) -> Tuple[List]:
    CLASSES  = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema',
                'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion',
                'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other',
                'Pneumonia', 'Pneumothorax', 'Support Devices']
        
    filenames_to_path = {path.split('/')[-1].split('.')[0]: path for path in paths}
    metadata = pd.read_csv(os.path.join(data_dir,'mimic-cxr-2.0.0-metadata.csv'))
    labels = pd.read_csv(os.path.join(data_dir,'mimic-cxr-2.0.0-chexpert.csv'))
    labels[CLASSES] = labels[CLASSES].fillna(0)
    labels = labels.replace(-1.0, 0.0)
    splits = pd.read_csv(os.path.join(data_dir,'mimic-cxr-ehr-split.csv'))
    metadata_with_labels = metadata.merge(labels[CLASSES+['study_id'] ], how='inner', on='study_id')
    filesnames_to_labels = dict(zip(metadata_with_labels['dicom_id'].values, metadata_with_labels[CLASSES].values))
    filenames_loaded = splits.loc[splits.split==split]['dicom_id'].values
    
    filenames_loaded = [filename  for filename in filenames_loaded if filename in filesnames_to_labels]

    return filenames_to_path, filenames_loaded, filesnames_to_labels

In [5]:
MIMIC_NORMALIZE ={"mean":torch.tensor([0.4723, 0.4723, 0.4723]), 
                  "std":torch.tensor([0.3023, 0.3023, 0.3023])}

train_transforms = T.Compose([T.Resize(256),
                              T.RandomHorizontalFlip(),
                              T.RandomAffine(degrees=45, scale=(.85, 1.15), shear=0, translate=(0.15, 0.15)),
                              T.CenterCrop(224),
                              T.ToTensor(),
                              T.Normalize(mean=MIMIC_NORMALIZE["mean"],
                                          std=MIMIC_NORMALIZE["std"])                                                                   
                            ])


val_test_transforms = T.Compose([T.Resize(256),
                                 T.CenterCrop(224),
                                 T.ToTensor(),
                                 T.Normalize(mean=MIMIC_NORMALIZE["mean"],
                                             std=MIMIC_NORMALIZE["std"])                                                                
                            ])

In [6]:
class MIMICCXR(Dataset):
    def __init__(self, 
                 paths: str,
                 data_dir: str, 
                 transform: Optional[T.Compose] = None, 
                 split: str = 'validate',
                 percentage:float = 1.0
                 ) -> None:
        self.data_dir = data_dir
        self.transform = transform
        self.filenames_to_path, \
        self.filenames_loaded, \
        self.filesnames_to_labels = preprocess(data_dir=self.data_dir,
                                               paths=paths,
                                               split=split
                                              )
        limit = (round(len(self.filenames_loaded) * percentage))
        self.filenames_loaded = self.filenames_loaded[0:limit]
 
        
    def __getitem__(self, index):
        if isinstance(index, str):
            img = Image.open(self.filenames_to_path[index]).convert('RGB')
            labels = torch.tensor(self.filesnames_to_labels[index]).float()

            if self.transform is not None:
                img = self.transform(img)
            return img, labels
        
        filename = self.filenames_loaded[index]
        
        img = Image.open(self.filenames_to_path[filename]).convert('RGB')

        labels = torch.tensor(self.filesnames_to_labels[filename]).float()
        
            
        if self.transform is not None:
            img = self.transform(img)
        return img, labels

    def __len__(self):
        return len(self.filenames_loaded)

In [7]:
data_dir = '/scratch/fs999/shamoutlab/data/physionet.org/files/mimic-cxr-jpg/2.0.0'

In [8]:
paths = glob.glob(os.path.join(data_dir,'resized','**','*.jpg'), recursive=True)
train_dataset = MIMICCXR(paths=paths,
                         data_dir= data_dir, 
                         split='train', 
                         transform=train_transforms,
                         )
val_dataset = MIMICCXR(paths=paths,
                        data_dir=data_dir, 
                        split='validate', 
                        transform=val_test_transforms,
                        )
test_dataset = MIMICCXR(paths=paths,
                        data_dir=data_dir, 
                        split='test', 
                        transform=val_test_transforms,
                        )

In [9]:
train_dataloader = DataLoader(dataset=train_dataset,
                             batch_size=64,
                             shuffle=True,
                             num_workers=16,
                             pin_memory=True,
                             drop_last=True
                             )

val_dataloader = DataLoader(dataset=val_dataset,
                             batch_size=64,
                             shuffle=False,
                             num_workers=16,
                             pin_memory=True,
                             drop_last=True
                             )

test_dataloader = DataLoader(dataset=test_dataset,
                             batch_size=64,
                             shuffle=False,
                             num_workers=16,
                             pin_memory=True,
                             drop_last=True
                             )

In [17]:
from torchvision.models.vision_transformer import VisionTransformer

In [18]:
backbone = VisionTransformer(image_size=224,
                             patch_size=16,
                             num_layers=12,
                             num_heads=6,
                             hidden_dim=192,
                             mlp_dim=192*4)

In [19]:
checkpoint_dir = '/scratch/sas10092/ChexMSN/models/lightning_logs/4178843-vit-tiny-lr=0.00000001/epoch=30-step=182249.ckpt'
all_weights = torch.load(checkpoint_dir,map_location='cpu')['state_dict']
# /scratch/sas10092/ChexMSN/models/lightning_logs/4181612-test-run-rand-cls/epoch=0-step=11757.ckpt

In [20]:
def parse_weights(weights: Dict[str,Tensor]) -> Dict[str,Tensor]:
    
    for k in list(weights.keys()):

        if k.startswith('model.target_backbone.'):
            
            if k.startswith('model.target_backbone') and not k.startswith('model.target_backbone.heads'):
                
                weights[k[len("model.target_backbone."):]] = weights[k]
                
        del weights[k]
    del weights['class_token'] 
    del weights['encoder.pos_embedding']    
    return weights

In [21]:
all_weights

OrderedDict([('model.backbone.class_token',
              tensor([[[-2.5591e-03,  1.2771e-03,  1.6636e-03,  4.5645e-05, -4.1930e-03,
                        -4.1763e-05, -1.0184e-03,  2.0642e-03, -3.4498e-03,  1.2950e-03,
                         3.1053e-03,  3.4107e-03,  2.5825e-03, -1.2213e-03,  1.7036e-03,
                        -1.1281e-03,  1.5405e-03,  2.0449e-03, -2.0113e-03, -2.3713e-03,
                        -1.3557e-04, -3.1177e-03,  3.2267e-03,  2.6641e-03,  1.2547e-03,
                        -3.0600e-03,  3.2708e-03, -2.1644e-04,  1.6515e-04,  2.1956e-03,
                        -1.5918e-03, -2.6223e-03,  1.2448e-03, -4.4017e-03, -1.1715e-03,
                        -1.4774e-03,  3.7219e-03,  8.7289e-04,  3.8992e-04,  4.9445e-04,
                        -1.9548e-03,  2.3612e-03, -1.6297e-03,  2.1119e-03,  1.3930e-03,
                         2.5902e-04, -8.5161e-04, -9.1866e-04,  1.1219e-03, -2.6997e-04,
                        -5.7893e-04, -1.3974e-04, -3.7005e-03,  7.

In [15]:
weight = parse_weights(all_weights)

In [21]:
backbone.load_state_dict(weight,strict=False)

_IncompatibleKeys(missing_keys=['class_token', 'encoder.pos_embedding', 'heads.head.weight', 'heads.head.bias'], unexpected_keys=[])

In [22]:
model = EvaluationModel(backbone=backbone,
                        learning_rate=0.0001,freeze=False)
# tiny running now

In [23]:
model.backbone.heads.head = nn.Linear(in_features=model.backbone.heads.head.in_features,
                                      out_features=model.output_dim)
model.backbone.heads.head

Linear(in_features=192, out_features=14, bias=True)

In [24]:
trainer = pl.Trainer(max_epochs=20,num_sanity_val_steps=0)

/home/sas10092/.conda/envs/chexmsn-env/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/sas10092/.conda/envs/chexmsn-env/lib/python3.9 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/sas10092/.conda/envs/chexmsn-env/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]

In [ ]:
trainer.fit(model=model, train_dataloaders=train_dataloader,val_dataloaders=val_dataloader)
trainer.test(model=model,dataloaders=test_dataloader)

You are using a CUDA device ('A100-PCIE-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type              | Params
-----------------------------------------------
0 | backbone | VisionTransformer | 5.5 M 
-----------------------------------------------
5.5 M     Trainable params
0         Non-trainable params
5.5 M     Total params
22.108    Total estimated model params size (MB)


Epoch 0: 100%|██████████| 5081/5081 [05:16<00:00, 16.05it/s, v_num=34]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  87%|████████▋ | 208/238 [00:06<00:00, 32.45it/s]


Epoch 1: 100%|██████████| 5081/5081 [05:17<00:00, 15.99it/s, v_num=34]    
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  74%|███████▍  | 177/238 [00:04<00:01, 37.11it/s]


Epoch 2: 100%|██████████| 5081/5081 [05:18<00:00, 15.96it/s, v_num=34]    
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  61%|██████▏   | 146/238 [00:04<00:02, 33.70it/s]


Epoch 3: 100%|██████████| 5081/5081 [05:18<00:00, 15.98it/s, v_num=34]    
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  49%|████▊     | 116/238 [00:03<00:03, 33.58it/s]


Epoch 4: 100%|██████████| 5081/5081 [05:18<00:00, 15.97it/s, v_num=34]    
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  36%|███▌      | 86/238 [00:02<00:04, 33.10it/s]


Epoch 5: 100%|██████████| 5081/5081 [05:18<00:00, 15.97it/s, v_num=34]    
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  23%|██▎       | 55/238 [00:01<00:06, 28.16it/s]


Epoch 6: 100%|██████████| 5081/5081 [05:18<00:00, 15.95it/s, v_num=34]    
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  10%|█         | 24/238 [00:00<00:06, 32.91it/s]


Validation DataLoader 0:  55%|█████▍    | 130/238 [00:04<00:04, 26.11it/s]


Epoch 7:  71%|███████   | 3600/5081 [03:45<01:32, 15.95it/s, v_num=34]    